* https://twitter.com/genkuroki/status/1401330514175291396
* https://github.com/genkuroki/public/blob/main/0001/harmonic%20numbers.ipynb
* https://twitter.com/genkuroki/status/1400995381933051904

In [1]:
versioninfo()
println()

function f(x, T=Float64)
    n = 1
    s = one(T)
    while s < x
        n += 1
        s += inv(T(n))
    end
    n, s
end

@time f(21)

Julia Version 1.7.0-DEV.1133
Commit db8d09609c (2021-05-21 13:48 UTC)
Platform Info:
  OS: Windows (x86_64-w64-mingw32)
  CPU: Intel(R) Core(TM) i7-10750H CPU @ 2.60GHz
  WORD_SIZE: 64
  LIBM: libopenlibm
  LLVM: libLLVM-11.0.1 (ORCJIT, skylake)
Environment:
  JULIA_DEPOT_PATH = D:\.julia
  JULIA_NUM_THREADS = 12
  JULIA_PYTHONCALL_EXE = D:\.julia\conda\3\python.exe

  0.646062 seconds


(740461601, 21.000000000402316)

In [2]:
using BenchmarkHistograms
@benchmark f(21) seconds=10

samples: 16; evals/sample: 1; memory estimate: 0 bytes; allocs estimate: 0
ns

 (6.45e8 - 6.5e8 ]  ██████████████████████████████ 9
 (6.5e8  - 6.54e8]  ██████████ 3
 (6.54e8 - 6.58e8]   0
 (6.58e8 - 6.62e8]  ██████▊2
 (6.62e8 - 6.66e8]  ██████▊2

                  Counts

min: 645.357 ms (0.00% GC); mean: 651.945 ms (0.00% GC); median: 649.090 ms (0.00% GC); max: 666.155 ms (0.00% GC).

In [3]:
run(`gcc --version`)
flush(stdout)

C_code = raw"""
long long f(double x) {
    long long n = 1;
    double s = 1.0;
    while (s < x) {
        n++;
        s += 1.0 / (double) n;
    }
    return n;
}
"""
display("text/markdown", "```C\n"*C_code*"\n```")

using Libdl
libname = tempname()
libname_dl = libname * "." * Libdl.dlext

open(`gcc -Wall -O3 -march=native -xc -shared -o $libname_dl -`, "w") do f
     print(f, C_code)
end
run(`ls -l $libname_dl`)
println()

f_gcc(x::Float64) = @ccall libname.f(x::Float64)::Int64
@time f_gcc(21.0)

gcc (Rev10, Built by MSYS2 project) 10.2.0
Copyright (C) 2020 Free Software Foundation, Inc.
This is free software; see the source for copying conditions.  There is NO
warranty; not even for MERCHANTABILITY or FITNESS FOR A PARTICULAR PURPOSE.



```C
long long f(double x) {
    long long n = 1;
    double s = 1.0;
    while (s < x) {
        n++;
        s += 1.0 / (double) n;
    }
    return n;
}

```

-rwxr-xr-x 1 genkuroki genkuroki 95812 Jun  6 09:56 C:\Users\GENKUR~1\AppData\Local\Temp\jl_oTBciIiDu6.dll

  0.671877 seconds


740461601

In [4]:
@benchmark f_gcc(21.0) seconds=10

samples: 16; evals/sample: 1; memory estimate: 0 bytes; allocs estimate: 0
ns

 (6.46e8 - 6.5e8 ]  ██████████████████████████████ 7
 (6.5e8  - 6.53e8]  █████████████████▎4
 (6.53e8 - 6.57e8]  ████████▋2
 (6.57e8 - 6.61e8]  ████████▋2
 (6.61e8 - 6.64e8]  ████▍1

                  Counts

min: 645.901 ms (0.00% GC); mean: 651.795 ms (0.00% GC); median: 649.763 ms (0.00% GC); max: 664.150 ms (0.00% GC).

In [5]:
0.67/740461601*6000125006293/60^2

1.5080997180929478

In [6]:
# Kahan-Babuska-Neumaier (KBN) algorithm
# See https://github.com/JuliaMath/KahanSummation.jl

versioninfo()
println()

function f_kbn(x, T=Float64)
    n = 1
    s = one(T)
    c = zero(T)
    while s < x
        n += 1
        a = inv(T(n))
        t = s + a
        c += abs(s) ≥ abs(a) ? ((s-t) + a) : ((a-t) + s)
        s = t
    end
    n, s + c
end

@time f_kbn(21)

Julia Version 1.7.0-DEV.1133
Commit db8d09609c (2021-05-21 13:48 UTC)
Platform Info:
  OS: Windows (x86_64-w64-mingw32)
  CPU: Intel(R) Core(TM) i7-10750H CPU @ 2.60GHz
  WORD_SIZE: 64
  LIBM: libopenlibm
  LLVM: libLLVM-11.0.1 (ORCJIT, skylake)
Environment:
  JULIA_DEPOT_PATH = D:\.julia
  JULIA_NUM_THREADS = 12
  JULIA_PYTHONCALL_EXE = D:\.julia\conda\3\python.exe

  1.318597 seconds


(740461601, 21.000000000401332)

In [7]:
using SpecialFunctions
H(n) = digamma(big(n+1)) + MathConstants.γ
setprecision(128) do; H(740461601) end

21.00000000040133333292517378163353387852

In [8]:
using BenchmarkHistograms
@benchmark f_kbn(21) seconds=20

samples: 16; evals/sample: 1; memory estimate: 0 bytes; allocs estimate: 0
ns

 (1.305e9 - 1.316e9]  ██████████████████████████████ 9
 (1.316e9 - 1.327e9]  ██████▊2
 (1.327e9 - 1.338e9]  ██████████ 3
 (1.338e9 - 1.349e9]  ███▍1
 (1.349e9 - 1.361e9]  ███▍1

                  Counts

min: 1.305 s (0.00% GC); mean: 1.320 s (0.00% GC); median: 1.313 s (0.00% GC); max: 1.361 s (0.00% GC).